In [1]:
%matplotlib inline
import graphlab
import graphlab.aggregate as agg
import numpy as np
import matplotlib.pyplot as plt  
import string
from nltk import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

stemmer = PorterStemmer()

plt.style.use('ggplot')

In [3]:
sf = graphlab.SFrame('./data/boardgames-no-comments')
sf.remove_columns(['age', 'average', 'bayesaverage', 'categories', 'maxplayers', 'maxplaytime', 'mechanics', 'minplayers', 'minplaytime', 'owned', 'playingtime', 'stddev', 'subdomains', 'trading', 'wanting', 'wishing', 'yearpublished'])
sf = sf[sf['usersrated'] > 10]
sf.head(1)

[INFO] This non-commercial license of GraphLab Create is assigned to djwelch666@googlemail.comand will expire on September 24, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-12728 - Server binary: c:\users\david\.virtualenvs\py\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\David\AppData\Local\Temp\graphlab_server_1445597723.log.0
[INFO] GraphLab Server Version: 1.6.1


description,id,name,usersrated
Marjapussi (Berry bag) isa marriage-style trick- ...,26721,Marjapussi,13


In [4]:
tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
def accept(w):
    return len(w) > 2 and w not in graphlab.text_analytics.stopwords()
sf['text'] = sf.select_column('description').apply(lambda x: ' '.join([stemmer.stem(w) for w in tokenizer.tokenize(BeautifulSoup(x.lower(), 'html.parser').get_text(strip=True)) if accept(w)]))

In [5]:
sf['word_count'] = graphlab.text_analytics.count_words(sf['text'])
sf['tfidf'] = graphlab.text_analytics.tf_idf(sf['word_count'])['docs']

In [11]:
tfidfs = sf[['tfidf']].stack('tfidf', new_column_name=['word','count'], drop_na=True)

In [ ]:
tfidfs.groupby(key_columns='word', operations={'count2': agg.MAX('count')}).sort('count2', ascending=True).print_rows(200)

In [6]:
model = graphlab.topic_model.create(sf['word_count'], num_topics=5, num_iterations=100, method='alias')

PROGRESS: Learning a topic model
PROGRESS:        Number of documents     21743
PROGRESS:            Vocabulary size     53683
PROGRESS:    Initializing topic assignments
PROGRESS:    Constructed alias samplers:0.199429s
PROGRESS:    Using 1 blocks.
PROGRESS: +-----------+---------------+----------------+-----------------+-------------------+
PROGRESS: | Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity | Elapsed for perp. |
PROGRESS: +-----------+---------------+----------------+-----------------+-------------------+
PROGRESS: | 10        | 3.78s         | 4.93842e+006   | 0               | 0                 |
PROGRESS: | 20        | 7.14s         | 5.14667e+006   | 0               | 0                 |
PROGRESS: | 30        | 10.41s        | 5.02557e+006   | 0               | 0                 |
PROGRESS: | 40        | 13.74s        | 4.9816e+006    | 0               | 0                 |
PROGRESS: | 50        | 16.91s        | 5.25754e+006   | 0               | 0         

In [7]:
model.summary()

Class                         : TopicModel

Schema
------
Vocabulary Size               : 53683
Topic Size                    : 53683

Settings
--------
Number of Topics              : 5
alpha                         : 10.0
beta                          : 0.1
Iterations                    : 100
Training time                 : 34.104
Verbose                       : False

Accessible fields             : 
m['topics']                   : An SFrame containing the topics.
m['vocabulary']               : An SArray containing the words in the vocabulary.
Useful methods                : 
m.get_topics()                : Get the most probable words per topic.
m.predict(new_docs)           : Make predictions for new documents.


In [8]:
model.get_topics().print_rows(50)

+-------+---------+------------------+
| topic |   word  |      score       |
+-------+---------+------------------+
|   0   |   game  | 0.0953980165261  |
|   0   |   play  | 0.0195153281091  |
|   0   |   time  |  0.012651598789  |
|   0   | victori | 0.00879427050664 |
|   0   |   edit  | 0.0080903784843  |
|   1   |  player |  0.047645218611  |
|   1   |  board  | 0.0303167017167  |
|   1   |   tile  | 0.0204664481291  |
|   1   |   move  | 0.0199332629939  |
|   1   |  build  | 0.0182790219334  |
|   2   |   game  | 0.0284588823084  |
|   2   |  power  | 0.0120060317398  |
|   2   | charact |  0.010081380663  |
|   2   |  expans | 0.00953314671993 |
|   2   |   set   | 0.00843376269576 |
|   3   |   card  |  0.106605932915  |
|   3   |  player | 0.0948037998154  |
|   3   |   play  | 0.0342006960134  |
|   3   |  point  | 0.0270936520914  |
|   3   |   game  | 0.0223903403405  |
|   4   |  battl  |  0.019666987404  |
|   4   |   war   | 0.0172350524694  |
|   4   |   rule  | 0.017

In [9]:
model.show()

Canvas is accessible via web browser at the URL: http://localhost:45986/index.html
Opening Canvas in default web browser.
